Here we use tximport::makeCountsFromAbundance function as it is described here https://docs.refine.bio/en/latest/main_text.html#processing-information

The goal is to preprocess our RNA-seq data in the same way as it is done in refine.bio.

After making lengthScaledTPM we also do quantile normalization on the reference distribution taken from here https://docs.refine.bio/en/latest/main_text.html#transformations
https://api.refine.bio/v1/qn_targets/HOMO_SAPIENS

In [1]:
library(tximport)
library(preprocessCore)

library(biomaRt)
library(tidyr)
library(dplyr)
library(curl)

In [3]:
listEnsemblArchives()
listMarts()
human.93 = useMart("ENSEMBL_MART_ENSEMBL", dataset="hsapiens_gene_ensembl", host="http://jul2018.archive.ensembl.org")  

name,date,url,version,current_release
<chr>,<chr>,<chr>,<chr>,<chr>
Ensembl GRCh37,Feb 2014,https://grch37.ensembl.org,GRCh37,
Ensembl 109,Feb 2023,https://feb2023.archive.ensembl.org,109,*
Ensembl 108,Oct 2022,https://oct2022.archive.ensembl.org,108,
Ensembl 107,Jul 2022,https://jul2022.archive.ensembl.org,107,
Ensembl 106,Apr 2022,https://apr2022.archive.ensembl.org,106,
Ensembl 105,Dec 2021,https://dec2021.archive.ensembl.org,105,
Ensembl 104,May 2021,https://may2021.archive.ensembl.org,104,
Ensembl 103,Feb 2021,https://feb2021.archive.ensembl.org,103,
Ensembl 102,Nov 2020,https://nov2020.archive.ensembl.org,102,


biomart,version
<chr>,<chr>
ENSEMBL_MART_ENSEMBL,Ensembl Genes 109
ENSEMBL_MART_MOUSE,Mouse strains 109
ENSEMBL_MART_SNP,Ensembl Variation 109
ENSEMBL_MART_FUNCGEN,Ensembl Regulation 109


Warning message:
“Ensembl will soon enforce the use of https.
Ensure the 'host' argument includes "https://"”


In [4]:
use_control <- FALSE # The notebook has to be launched for TRUE and then FALSE values of this variable

In [5]:
if (use_control) {
    our_table_name <- "../data/ours_counts/h.gene.count.csv"
    path_for_saving <- "../data/intermediate/control_lengthScaledTPM.csv"
    path_for_saving_normalized <- "../data/intermediate/control_lengthScaledTPM_normalized.csv"
} else {
    our_table_name <- "../data/ours_counts/schizo_gene_count_matrix.csv"
    path_for_saving <- "../data/intermediate/sz_lengthScaledTPM.csv"
    path_for_saving_normalized <- "../data/intermediate/sz_lengthScaledTPM_normalized.csv"
}

In [6]:
counts <- read.csv(our_table_name)

In [7]:
gene.data = data.frame(gene = counts[,1])
counts = counts[,-1]

# Get TPM

In [9]:
gene_coords=getBM(
    attributes=c(#"hgnc_symbol",
        "ensembl_gene_id", 
        "transcript_length",
        "gene_biotype"
    ), 
    filters=c("ensembl_gene_id"), 
    values=as.character(gene.data$gene), 
    mart=human.93,
    useCache = FALSE
)


Batch submitting query [==>----------------------------]   8% eta:  8m

Batch submitting query [====>--------------------------]  17% eta:  4m

Batch submitting query [=======>-----------------------]  25% eta:  3m

Batch submitting query [=========>---------------------]  33% eta:  2m

Batch submitting query [============>------------------]  42% eta:  2m

Batch submitting query [===============>---------------]  50% eta:  1m

Batch submitting query [=================>-------------]  58% eta:  1m

Batch submitting query [====================>----------]  67% eta: 45s

Batch submitting query [======================>--------]  75% eta: 31s

Batch submitting query [=========================>-----]  83% eta: 20s

Batch submitting query [===========================>---]  92% eta: 10s
                                                                      



In [10]:
gene.data.ens = gene_coords %>% 
dplyr::filter(gene_biotype %in% c("protein_coding")) %>% #,"lincRNA","antisense"
dplyr::select(ensembl_gene_id, gene_biotype, transcript_length) %>%
dplyr::group_by(ensembl_gene_id) %>%
dplyr::summarise(mean = mean(transcript_length),
                 median = median(transcript_length),
                 longest_isoform = max(transcript_length))
nrow(gene.data.ens)

[1] 19912

In [11]:
gene.data.ens = arrange(gene.data.ens, ensembl_gene_id)
counts = counts[order(gene.data),]
gene.data = gene.data[order(gene.data),]

Warning message in xtfrm.data.frame(x):
“cannot xtfrm data frames”
Warning message in xtfrm.data.frame(x):
“cannot xtfrm data frames”


In [12]:
coding = counts[gene.data %in% gene.data.ens$ensembl_gene_id,]+1 # select protein-coding genes 
coding = apply(coding, 2, function(x) 1000*(x/gene.data.ens$mean)) # get RPK - reads per kilobase
coding = apply(coding, 2, function(x) 1000000*(x/sum(x))) # get TPM - dividing by "per milion" factor

In [13]:
# selected.genes = gene.data[gene.data %in% gene.data.ens$ensembl_gene_id][rowMeans(coding) > 1]# select genes with mean TPM > 1 <=> mean log > 0
# coding = coding[rowMeans(coding) > 1,] # select genes with mean TPM > 1 <=> mean log > 0 
# length(selected.genes)
# coding = log10(coding)
# coding = apply(coding, 2, function(x) x-median(x))

# Get lengthScaledTPM

In [14]:
lengthScaledTPM <- tximport::makeCountsFromAbundance(
    countsMat=counts,
    abundanceMat=coding,
    lengthMat=gene.data.ens[,2],
    countsFromAbundance="lengthScaledTPM"
)

In [15]:
write.csv(lengthScaledTPM, path_for_saving, row.names=gene.data.ens$ensembl_gene_id)

# Quantile Normalization

In [16]:
reference_distr <- read.csv("../data/refine.bio/reference_distr.tsv", sep='\t', header=FALSE)

In [17]:
lengthScaledTPM_normalized <- normalize.quantiles.use.target(lengthScaledTPM, target=as.vector(t(reference_distr)))

In [18]:
write.csv(lengthScaledTPM_normalized, path_for_saving_normalized, row.names=gene.data.ens$ensembl_gene_id)

In [19]:
sessionInfo()

R version 4.2.2 Patched (2022-11-10 r83330)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.10.0
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.10.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=ru_RU.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=ru_RU.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=ru_RU.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=ru_RU.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] curl_5.0.0            ggplot2_3.4.1         dplyr_1.1.0          
[4] tidyr_1.3.0           biomaRt_2.54.0        preprocessCore_1.60.2
[7] tximport_1.26.1      

loaded via a namespace (and not attached):
 [1] prettyunits_1.1.1      png_0.1-8    